In [1]:
import pandas as pd
import pyspark
import os 
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf      # sf = spark functions
import pyspark.sql.types as st          # st = spark types
import datetime as dt

spark = SparkSession.builder.master('local').getOrCreate()

# spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/02/14 17:01:38 WARN Utils: Your hostname, DESKTOP-SU1U0NI resolves to a loopback address: 127.0.1.1; using 172.21.244.112 instead (on interface eth0)
23/02/14 17:01:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/14 17:01:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [51]:
data_dir = './data'

cpi_file = 'US-CPI.csv'
unemp_file = 'USUnemployment.csv'

c_schema = 'Yearmon string, CPI float'
u_schema = 'Year int, Jan float ,Feb float ,Mar float,Apr float ,May float,Jun float ,Jul float ,Aug float ,Sep float,Oct float,Nov float,Dec float'

cpi_df = spark.read.csv(os.path.join(data_dir, cpi_file), schema=c_schema, header=True, enforceSchema=True)
unemp_df = spark.read.csv(os.path.join(data_dir, unemp_file), schema=u_schema, header=True, enforceSchema=True)

cpi_df.show(5)
unemp_df.show(5)

+----------+---+
|   Yearmon|CPI|
+----------+---+
|01-01-1913|9.8|
|01-02-1913|9.8|
|01-03-1913|9.8|
|01-04-1913|9.8|
|01-05-1913|9.7|
+----------+---+
only showing top 5 rows

+----+---+---+---+---+---+---+---+---+---+---+---+---+
|Year|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|
+----+---+---+---+---+---+---+---+---+---+---+---+---+
|1948|3.4|3.8|4.0|3.9|3.5|3.6|3.6|3.9|3.8|3.7|3.8|4.0|
|1949|4.3|4.7|5.0|5.3|6.1|6.2|6.7|6.8|6.6|7.9|6.4|6.6|
|1950|6.5|6.4|6.3|5.8|5.5|5.4|5.0|4.5|4.4|4.2|4.2|4.3|
|1951|3.7|3.4|3.4|3.1|3.0|3.2|3.1|3.1|3.3|3.5|3.5|3.1|
|1952|3.2|3.1|2.9|2.9|3.0|3.0|3.2|3.4|3.1|3.0|2.8|2.7|
+----+---+---+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [58]:
from pyspark.sql.functions import col,avg
# rename cols in cpi df

for column in cpi_df.columns: 
  cpi_df = cpi_df.withColumnRenamed(column, column.lower())

cpi_df = cpi_df.withColumnRenamed('yearmon', 'year')

# split year column into month, date, year columns
split_cpi_df = cpi_df.withColumn("month", sf.split(sf.col('year'), "-").getItem(0))\
              .withColumn('date', sf.split(sf.col('year'), "-").getItem(1))\
              .withColumn('year', sf.split(sf.col('year'), "-").getItem(2))

# cast year col to int for groupby method
# cpi_df = cpi_df.withColumn('year', sf.col('year').cast('int'))

# rename cols in unemployment df
for column in unemp_df.columns: 
  unemp_df = unemp_df.withColumnRenamed(column, column.lower())

# calculate avg unemployment rate per year
udf_avg = sf.udf(lambda array: sum(array)/len(array))
unemp_df = unemp_df.withColumn("avg_unemp_rate_per_year", udf_avg(sf.array('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')).cast('float'))

# calculate avg inflation rate per year
avg_cpi_df = split_cpi_df.groupBy(sf.col('year')) \
                .agg(sf.avg(sf.col('cpi')))

# avg_cpi_df.show(truncate=False)
cpi_df.printSchema()
split_cpi_df.show(5)
avg_cpi_df.show(5)

root
 |-- year: string (nullable = true)
 |-- cpi: float (nullable = true)

+----+---+-----+----+
|year|cpi|month|date|
+----+---+-----+----+
|1913|9.8|   01|  01|
|1913|9.8|   01|  02|
|1913|9.8|   01|  03|
|1913|9.8|   01|  04|
|1913|9.7|   01|  05|
+----+---+-----+----+
only showing top 5 rows

+----+------------------+
|year|          avg(cpi)|
+----+------------------+
|1953|26.766666571299236|
|1957|28.091666380564373|
|1987|           113.625|
|1956|27.183333079020183|
|1936|13.866666634877523|
+----+------------------+
only showing top 5 rows



In [36]:
# read and rename cpi file / parse_dates=['Yearmon']
cpi  = pd.read_csv(os.path.join(data_dir, cpi_file), header=0)
cpi = cpi.rename(columns={'Yearmon': 'year', 'CPI': 'cpi'})
cpi[['month', 'date', 'year']] = cpi.year.str.split("-", expand=True)

# read and rename unemployment file
unemp = pd.read_csv(os.path.join(data_dir, unemp_file), header=0)
unemp.columns = unemp.columns.str.lower()


# calculate the avg yearly unemp. rate
columns = ['jan',	'feb',	'mar',	'apr',	'may'	,'jun',	'jul',	'aug',	'sep',	'oct',	'nov',	'dec']
unemp['avg_unemp_per_year']  = unemp[columns].mean(axis=1)
avg_unemp_per_year  = unemp[['year', 'avg_unemp_per_year']]

# calculate the avg yearly cpi rate
avg_cpi_per_year = cpi.groupby('year').mean('cpi')
avg_cpi_per_year = avg_cpi_per_year.reset_index()

avg_unemp_per_year.head(5)
avg_cpi_per_year.head(10)
# unemp.dtypes


,year,cpi
0,1913,9.883333
1,1914,10.016667
2,1915,10.108333
3,1916,10.883333
4,1917,12.825000
5,1918,15.041667
6,1919,17.333333
7,1920,20.041667
8,1921,17.850000
9,1922,16.750000


In [ ]:
from google.cloud import bigquery

# create bigquery client
client = bigquery.Client()

# create full table id
project_id = "team-week-3"
dataset_id = "tech_stocks_world_events"

cpi_table_name = "cpi_rates"
unemp_table_name = "unemployment_rates"

# table_id = f"{project_id}.{dataset_id}.{table_name}"


# create bigquery client
client = bigquery.Client()

# create our own bigquery schema
avg_cpi_schema = [
    bigquery.SchemaField("year", "INT", mode="REQUIRED"),
    bigquery.SchemaField("avg_cpi_per_year", "FLOAT", mode="REQUIRED")]

avg_emp_schema = [
    bigquery.SchemaField("year", "INT", mode="REQUIRED"),
    bigquery.SchemaField("avg_cpi_per_year", "FLOAT", mode="REQUIRED")]


